# NLP HW4: Question Answering

Aryan Ahadinia, Arshan Dalili, MohammadMahdi Abootorabi

In [ ]:
! pip install transformers datasets evaluate

In [ ]:
from datasets import load_dataset
from evaluate import load
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import pipeline
from tqdm import tqdm

In this part, we load `SajjadAyoubi/persian_qa` dataset from huggingface hub. This dataset contains train and test parts itself.

In [ ]:
persian_qa = load_dataset("SajjadAyoubi/persian_qa")
persian_qa

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/6bed7cf0f82631fb5231ba39906ba835ef73fb005b6e7371869afd01e5f98ddc


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/3b640ed1f99bd004e2d515cf66938e75cd85d8f359739d9ba644ed3dc533b849
Dataset persian_qa downloaded and prepared to /root/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [ ]:
persian_qa["train"][0]

{'id': 1,
 'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'questio

Now, we drop records without possible answers in the context.

In [ ]:
persian_qa["train"] = persian_qa["train"].filter(lambda e: e["answers"]["answer_start"])
persian_qa["validation"] = persian_qa["validation"].filter(lambda e: e["answers"]["answer_start"])
persian_qa

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 6306
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 651
    })
})

Here we load `ForutanRad/bert-fa-QA-v1` QA model and instanciate a tokenizer from it.

In [ ]:
model_name = "ForutanRad/bert-fa-QA-v1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Now we prepocess our data in order to prepare it for training.

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_persian_qa = persian_qa.map(
    preprocess_function,
    batched=True,
    remove_columns=persian_qa["train"].column_names
)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
data_collator = DefaultDataCollator()
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Now we use trainer to fine tune our model in 5 epochs.

In [ ]:
training_args = TrainingArguments(
    output_dir="qa_answering_fine_tuned",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_persian_qa["train"],
    eval_dataset=tokenized_persian_qa["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6306
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1975
  Number of trainable parameters = 162252290


Epoch,Training Loss,Validation Loss
1,No log,2.089276
2,1.087400,2.117393
3,0.971800,2.136316
4,0.907200,2.187001
5,0.907200,2.193246


***** Running Evaluation *****
  Num examples = 651
  Batch size = 16
Saving model checkpoint to qa_answering_fine_tuned/checkpoint-500
Configuration saved in qa_answering_fine_tuned/checkpoint-500/config.json
Model weights saved in qa_answering_fine_tuned/checkpoint-500/pytorch_model.bin
tokenizer config file saved in qa_answering_fine_tuned/checkpoint-500/tokenizer_config.json
Special tokens file saved in qa_answering_fine_tuned/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 651
  Batch size = 16
Saving model checkpoint to qa_answering_fine_tuned/checkpoint-1000
Configuration saved in qa_answering_fine_tuned/checkpoint-1000/config.json
Model weights saved in qa_answering_fine_tuned/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in qa_answering_fine_tuned/checkpoint-1000/tokenizer_config.json
Special tokens file saved in qa_answering_fine_tuned/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples =

TrainOutput(global_step=1975, training_loss=0.958435321276701, metrics={'train_runtime': 2339.1082, 'train_samples_per_second': 13.479, 'train_steps_per_second': 0.844, 'total_flos': 6179015054914560.0, 'train_loss': 0.958435321276701, 'epoch': 5.0})

Now we test our model with one of validation records.

In [ ]:
question = persian_qa["validation"][0]['question']
context = persian_qa["validation"][0]['context']

In [ ]:
question

'پایتخت اسپانیا کجاست؟'

In [ ]:
context

'باشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق\u200cترین تیم تاریخ فوتبال اسپانیا و موفق\u200cترین تیم تاریخ فوتبال اروپا و موفق\u200cترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن\u200cها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه\u200cهای جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده\u200cاست. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر روی آرم این باشگاه قرار گرفت. رئال مادرید سال\u200cهاست که در لالیگا، از تیم\u200cهای قدرتمند محسوب می\u200cشود و رقیب اصلی این تیم نیز، بارسلونا است. از نظر درآمد، رئال مادرید با ۵۵۷ میلیون یورو درآمد در ف

In [ ]:
model= model.to('cpu')

In [ ]:
question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)
question_answerer(question=question, context=context)

{'score': 0.9445521831512451, 'start': 61, 'end': 67, 'answer': 'مادرید'}

Now we want to calculate EM and F1 using huggingface `squad` metric. First, we adapt out data to required format and then we use this library to calculate F1 and EM.

In [ ]:
model = model.to('cpu')
predictions = []
for i in tqdm(range(len(persian_qa["validation"]))):
    question = persian_qa["validation"][i]["question"]
    context = persian_qa["validation"][i]["context"]
    prediction = question_answerer(question=question, context=context)
    predictions.append(prediction)

100%|██████████| 651/651 [06:35<00:00,  1.65it/s]


In [ ]:
eval_predictions = [{"prediction_text": pred['answer'], "id": str(i)} for i, pred in enumerate(predictions)]
eval_predictions[:10]

[{'prediction_text': 'مادرید', 'id': '0'},
 {'prediction_text': 'فیفا', 'id': '1'},
 {'prediction_text': '۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی',
  'id': '2'},
 {'prediction_text': '«سلطنتی»', 'id': '3'},
 {'prediction_text': 'مادرید، پایتخت اسپانیا', 'id': '4'},
 {'prediction_text': 'مادرید، پایتخت اسپانیا', 'id': '5'},
 {'prediction_text': 'بارسلونا', 'id': '6'},
 {'prediction_text': 'باشگاه فوتبال بارسلونا که بیشتر بارسلونا یا به صورت مخفف بارسا شناخته می\u200cشود',
  'id': '7'},
 {'prediction_text': 'اسپانیایی', 'id': '8'},
 {'prediction_text': 'منطقهٔ کاتالونیای اسپانیا', 'id': '9'}]

In [ ]:
eval_references = [{"answers": ref["answers"], "id": str(i)} for i, ref in enumerate(persian_qa["validation"])]
eval_references[:10]

[{'answers': {'text': ['مادرید', 'مادرید'], 'answer_start': [19, 19]},
  'id': '0'},
 {'answers': {'text': ['فیفا', 'به انتخاب فیفا'], 'answer_start': [218, 208]},
  'id': '1'},
 {'answers': {'text': ['۱۳', '۱۳'], 'answer_start': [329, 329]}, 'id': '2'},
 {'answers': {'text': ['سلطنتی', 'سلطنتی'], 'answer_start': [648, 648]},
  'id': '3'},
 {'answers': {'text': ['مادرید، پایتخت اسپانیا', 'اسپانیا'],
   'answer_start': [61, 76]},
  'id': '4'},
 {'answers': {'text': ['شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد',
    'شاه آلفونسو سیزدهم'],
   'answer_start': [672, 672]},
  'id': '5'},
 {'answers': {'text': ['رقیب اصلی این تیم نیز، بارسلونا است', 'بارسلونا'],
   'answer_start': [857, 880]},
  'id': '6'},
 {'answers': {'text': ['بارسلونا', 'باشگاه فوتبال بارسلونا'],
   'answer_start': [14, 0]},
  'id': '7'},
 {'answers': {'text': ['اسپانیا', 'اسپانیا'], 'answer_start': [103, 103]},
  'id': '8'},
 {'answers': {'text': ['منطقهٔ کاتالونیای', 'کاتالونیا'],
   'answer_start': [138, 145]},
  

In [ ]:
squad_metric = load("squad")

In [ ]:
results = squad_metric.compute(predictions=eval_predictions, references=eval_references)
results

{'exact_match': 43.93241167434716, 'f1': 68.7953555467024}

In [ ]:
trainer.save_model("Finetuned_QA")

Saving model checkpoint to Finetuned_QA
Configuration saved in Finetuned_QA/config.json
Model weights saved in Finetuned_QA/pytorch_model.bin
tokenizer config file saved in Finetuned_QA/tokenizer_config.json
Special tokens file saved in Finetuned_QA/special_tokens_map.json


In [ ]:
!zip /content/drive/MyDrive/Finetuned_QA.zip Finetuned_QA/*

  adding: Finetuned_QA/config.json (deflated 47%)
  adding: Finetuned_QA/pytorch_model.bin (deflated 8%)
  adding: Finetuned_QA/special_tokens_map.json (deflated 42%)
  adding: Finetuned_QA/tokenizer_config.json (deflated 44%)
  adding: Finetuned_QA/tokenizer.json (deflated 72%)
  adding: Finetuned_QA/training_args.bin (deflated 49%)
  adding: Finetuned_QA/vocab.txt (deflated 62%)
